In [2]:
import torch

if torch.cuda.is_available():
    print('Default CUDA Device: {}'.format(torch.cuda.get_device_name()))
else:
    print("No GPU Found. Using CPU instead.")

Default CUDA Device: Tesla V100-SXM2-16GB


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Batch Processing Part 1

In [ ]:
import numpy as np
import torch
import pandas as pd
import seaborn as sns
import bz2
import re
import os
import gc
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import TrainingArguments, Trainer

train_file = bz2.BZ2File('/content/drive/MyDrive/data_mining/train.ft.txt.bz2')
test_file = bz2.BZ2File('/content/drive/MyDrive/data_mining/test.ft.txt.bz2')

def load_extract(file):
    texts, labels = [], [] 
    for line in file:
        x = line.decode('utf-8')  # decode binary to string
        labels.append(int(x[9]) - 1)  # extract labels
        texts.append(x[10:].strip())  # extract texts
    print('Done !')
    return np.array(labels), texts

train_labels, train_texts = load_extract(train_file)
test_labels, test_texts = load_extract(test_file)

train_data = pd.DataFrame(data=train_labels, columns=['label'])
train_data['text'] = train_texts

unique_labels = train_data['label'].unique()

# Initialize an empty DataFrame to store your filtered data
bert_train = pd.DataFrame()

# Loop through each label and select 5000 random records
for label in unique_labels:
    label_data = train_data.loc[train_data['label'] == label].sample(n=500000, random_state=42)
    bert_train = pd.concat([bert_train, label_data])

# Save the filtered data to a new file
#bert_train.to_csv('bert_train.csv', index=False)
dataset = bert_train[['text', 'label']]

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model = model.to('cuda')

X = list(dataset["text"])
y = list(dataset["label"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

args = TrainingArguments(
    output_dir="output",
    num_train_epochs=2,
    per_device_train_batch_size=64

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)
trainer.train()

# Batch Processing Part 2

In [23]:
import numpy as np
import torch
import pandas as pd
import seaborn as sns
import bz2
import re
import os
import gc
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import TrainingArguments, Trainer

In [24]:
train_file = bz2.BZ2File('/content/drive/MyDrive/data_mining/train.ft.txt.bz2')
test_file = bz2.BZ2File('/content/drive/MyDrive/data_mining/test.ft.txt.bz2')

def load_extract(file):
    texts, labels = [], [] 
    for line in file:
        x = line.decode('utf-8')  # decode binary to string
        labels.append(int(x[9]) - 1)  # extract labels
        texts.append(x[10:].strip())  # extract texts
    print('Done !')
    return np.array(labels), texts

In [25]:
train_labels, train_texts = load_extract(train_file)
test_labels, test_texts = load_extract(test_file)

train_data = pd.DataFrame(data=train_labels, columns=['label'])
train_data['text'] = train_texts

unique_labels = train_data['label'].unique()



Done !
Done !


In [30]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model = model.to('cuda')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.w

In [27]:
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=2,
    per_device_train_batch_size=64
)

In [28]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [29]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [11]:
for i in range(20):
    print(f"Training batch {i+1}")
    # Load model from previous checkpoint
    if i > 0:
        model = DistilBertForSequenceClassification.from_pretrained(f"/content/drive/MyDrive/bert_checkpoints/model_batch_{i}")
        model.to('cuda')
    # Initialize an empty DataFrame to store your filtered data
    bert_train = pd.DataFrame()

    # Loop through each label and select 100000 random records
    for label in unique_labels:
        label_data = train_data.loc[train_data['label'] == label].sample(n=50000, random_state=42*i)
        bert_train = pd.concat([bert_train, label_data])

    # Create dataset
    dataset = bert_train[['text', 'label']]
    X = list(dataset["text"])
    y = list(dataset["label"])
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
    X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=256)
    X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=256)
    train_dataset = Dataset(X_train_tokenized, y_train)
    val_dataset = Dataset(X_val_tokenized, y_val)

    # Create Trainer instance and train the model
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )
    trainer.train()

    # Save the model
    model.save_pretrained(f"/content/drive/MyDrive/bert_checkpoints/model_batch_{i+1}")

    # Free up storage
    del bert_train
    del dataset
    del X_train
    del X_val
    del y_train
    del y_val
    del X_train_tokenized
    del X_val_tokenized
    del train_dataset
    del val_dataset
    gc.collect()

Training batch 1


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


Training batch 2


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


Training batch 5


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


# Complete Dataset Fine-tuning

In [31]:
# Split the data into batches of 50,000
batch_size = 50000
num_batches = len(train_data) // batch_size
print("Total Batches: ", num_batches)
batches = [train_data.iloc[i*batch_size:(i+1)*batch_size] for i in range(num_batches)]

# Train the model on each batch
for i, batch in enumerate(batches):
    print(f"Training batch {i+1}")
    
    # Load model from previous checkpoint
    if i > 0:
        model = DistilBertForSequenceClassification.from_pretrained(f"/content/drive/MyDrive/bert_checkpoints/model_batch_{i}")
        model.to('cuda')
    
    # Create dataset
    dataset = batch[['text', 'label']]
    X = list(dataset["text"])
    y = list(dataset["label"])
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
    X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=256)
    X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=256)
    train_dataset = Dataset(X_train_tokenized, y_train)
    val_dataset = Dataset(X_val_tokenized, y_val)

    # Create Trainer instance and train the model
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )
    trainer.train()

    # Save the model
    model.save_pretrained(f"/content/drive/MyDrive/bert_checkpoints/model_batch_{i+1}/")

    # Free up storage
    del dataset
    del X_train
    del X_val
    del y_train
    del y_val
    del X_train_tokenized
    del X_val_tokenized
    del train_dataset
    del val_dataset
    gc.collect()


Total Batches:  72
Training batch 1


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.195600
1000,0.104700


Training batch 2


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

Training batch 62


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.104700
1000,0.063300


Training batch 63


KeyboardInterrupt: ignored

In [33]:
trainer.evaluate() # complete model

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.11848532408475876,
 'eval_accuracy': 0.9648,
 'eval_precision': 0.9692937563971341,
 'eval_recall': 0.9590844642495443,
 'eval_f1': 0.9641620851150479,
 'eval_runtime': 20.0972,
 'eval_samples_per_second': 497.581,
 'eval_steps_per_second': 62.198,
 'epoch': 2.0}

In [ ]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.09618774056434631,
 'eval_accuracy': 0.96765,
 'eval_precision': 0.969480975805642,
 'eval_recall': 0.9657,
 'eval_f1': 0.967586794248785,
 'eval_runtime': 48.1861,
 'eval_samples_per_second': 415.058,
 'eval_steps_per_second': 51.882,
 'epoch': 1.0}

In [ ]:
!zip -r /content/sentiment-analysis-model-updated.zip /content/model_batch_20

  adding: content/model_batch_20/ (stored 0%)
  adding: content/model_batch_20/pytorch_model.bin (deflated 8%)
  adding: content/model_batch_20/config.json (deflated 45%)


In [ ]:
from google.colab import files
files.download("/content/sentiment-analysis-model-updated.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
model.save_pretrained('/content/drive/MyDrive/models/')

In [ ]:
trainer.save_model('sentiment-analysis-model')

In [ ]:
import numpy as np
import torch
import pandas as pd
import seaborn as sns
import bz2
import re
import os
import gc
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
train_file = bz2.BZ2File('train.ft.txt.bz2')
test_file = bz2.BZ2File('test.ft.txt.bz2')

In [ ]:
def load_extract(file):
    texts, labels = [], [] 
    for line in file:
        x = line.decode('utf-8')  # decode binary to string
        labels.append(int(x[9]) - 1)  # extract labels
        texts.append(x[10:].strip())  # extract texts
    print('Done !')
    return np.array(labels), texts

In [ ]:
train_labels, train_texts = load_extract(train_file)
test_labels, test_texts = load_extract(test_file)

Done !
Done !


In [ ]:
train_data = pd.DataFrame(data=train_labels, columns=['label'])
train_data['text'] = train_texts

In [ ]:
test_data = pd.DataFrame(data=test_labels, columns=['label'])
test_data['text'] = test_texts

In [ ]:
test_data.head()

,label,text
0,1,Great CD: My lovely Pat has one of the GREAT v...
1,1,One of the best game music soundtracks - for a...
2,0,Batteries died within a year ...: I bought thi...
3,1,"works fine, but Maha Energy is better: Check o..."
4,1,Great for the non-audiophile: Reviewed quite a...


In [ ]:
train_data.head()

,label,text
0,1,Stuning even for the non-gamer: This sound tra...
1,1,The best soundtrack ever to anything.: I'm rea...
2,1,Amazing!: This soundtrack is my favorite music...
3,1,Excellent Soundtrack: I truly like this soundt...
4,1,"Remember, Pull Your Jaw Off The Floor After He..."


In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
train_data["label"].value_counts()

1    1800000
0    1800000
Name: label, dtype: int64

In [ ]:
unique_labels = train_data['label'].unique()

# Initialize an empty DataFrame to store your filtered data
bert_train = pd.DataFrame()

# Loop through each label and select 5000 random records
for label in unique_labels:
    label_data = train_data.loc[train_data['label'] == label].sample(n=500000, random_state=42)
    bert_train = pd.concat([bert_train, label_data])

# Save the filtered data to a new file
bert_train.to_csv('bert_train.csv', index=False)

In [ ]:
import os
print(os.getcwd())

/content


In [ ]:
print(os.listdir())

['.config', 'train.ft.txt.bz2', 'test.ft.txt.bz2', 'bert_train.csv', 'sample_data']


In [ ]:
dataset = pd.read_csv("bert_train.csv")

In [ ]:
dataset['label'].value_counts()

1    500000
0    500000
Name: label, dtype: int64

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
# from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model = model.to('cuda')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classi

In [ ]:
longest_string_length = len(max(dataset['text'], key=len))
longest_string_length

1015

In [ ]:
dataset['text_length'] = dataset['text'].apply(len)

# calculate the mean of the text_length column
mean_text_length = dataset['text_length'].mean()

# print the result
print(mean_text_length)

431.506289


In [ ]:
dataset = dataset[['text', 'label']]

In [ ]:
dataset.shape

(1000000, 2)

In [ ]:
X = list(dataset["text"])
y = list(dataset["label"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

NameError: ignored

In [ ]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
len(X_train)

80000

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [ ]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

args = TrainingArguments(
    output_dir="output",
    num_train_epochs=2,
    per_device_train_batch_size=64

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.195800
1000,0.149900
1500,0.111000
2000,0.077700
2500,0.076600


TrainOutput(global_step=2500, training_loss=0.1221945556640625, metrics={'train_runtime': 4512.4671, 'train_samples_per_second': 35.457, 'train_steps_per_second': 0.554, 'total_flos': 1.399186898304e+16, 'train_loss': 0.1221945556640625, 'epoch': 2.0})

# New Section

In [ ]:
trainer.evaluate() # for 2 epochs with 100k records

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.14845658838748932,
 'eval_accuracy': 0.95365,
 'eval_precision': 0.9539677774442109,
 'eval_recall': 0.9533,
 'eval_f1': 0.9536337718201371,
 'eval_runtime': 271.246,
 'eval_samples_per_second': 73.734,
 'eval_steps_per_second': 9.217,
 'epoch': 2.0}

In [ ]:
trainer.evaluate() # for 10 epochs

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.6994720101356506,
 'eval_accuracy': 0.9285,
 'eval_precision': 0.9246778989098117,
 'eval_recall': 0.933,
 'eval_f1': 0.9288203086112494,
 'eval_runtime': 19.4377,
 'eval_samples_per_second': 102.893,
 'eval_steps_per_second': 12.862,
 'epoch': 10.0}

In [ ]:
trainer.evaluate() # for 1 epoch

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.21313944458961487,
 'eval_accuracy': 0.9375,
 'eval_precision': 0.9441624365482234,
 'eval_recall': 0.93,
 'eval_f1': 0.9370277078085644,
 'eval_runtime': 19.8774,
 'eval_samples_per_second': 100.617,
 'eval_steps_per_second': 12.577,
 'epoch': 1.0}

In [ ]:
print("hi")

hi


In [ ]:
#text = "That was good product"
text = "go to hell"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.9487, -0.9959]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.8748, 0.1252]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


array([[0.8748474 , 0.12515257]], dtype=float32)

In [ ]:
trainer.save_model('sentiment-analysis-model')

In [ ]:
!zip -r /content/sentiment-analysis-model.zip /content/sentiment-analysis-model

  adding: content/sentiment-analysis-model/ (stored 0%)
  adding: content/sentiment-analysis-model/training_args.bin (deflated 49%)
  adding: content/sentiment-analysis-model/pytorch_model.bin (deflated 8%)
  adding: content/sentiment-analysis-model/config.json (deflated 46%)


In [ ]:
from google.colab import files
files.download("/content/sentiment-analysis-model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
os.listdir()

['.config',
 'sentiment-analysis-model',
 'train.ft.txt.bz2',
 'test.ft.txt.bz2',
 'bert_train.csv',
 'output',
 'sample_data']

In [ ]:
import os
print(os.getcwd())

/content


In [ ]:
print(os.listdir())

['.config', 'colab_files.zip', 'sentiment-analysis-model', 'sentiment-analysis-model.zip', 'train.ft.txt.bz2', 'test.ft.txt.bz2', 'bert_train.csv', 'output', 'sample_data']


In [ ]:
from transformers.models.distilbert.modeling_distilbert import DistilBertForSequenceClassification
pretrained_model = DistilBertForSequenceClassification.from_pretrained('sentiment-analysis-model')
pretrained_model.to('cuda')

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
#text = "Despite the initial skepticism surrounding its effectiveness, the new drug has shown promising results in treating a wide range of symptoms associated with the rare neurological disorder, which has left many patients and their families hopeful for the future."
text = "While the product itself was impressive in terms of its versatility and durability, the instructions provided in the manual were convoluted and lacked clarity, making the assembly process much more challenging than it needed to be. Additionally, the customer service team was slow to respond to my inquiries and provided unhelpful solutions, leaving me feeling frustrated and dissatisfied with my overall experience."
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = pretrained_model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
predictions

array([[9.9999714e-01, 2.8098161e-06]], dtype=float32)

In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
! cd content

/bin/bash: line 0: cd: content: No such file or directory


In [ ]:
!unzip sentiment-analysis-model.zip

Archive:  sentiment-analysis-model.zip
   creating: content/sentiment-analysis-model/
  inflating: content/sentiment-analysis-model/training_args.bin  
  inflating: content/sentiment-analysis-model/pytorch_model.bin  
  inflating: content/sentiment-analysis-model/config.json  


In [ ]:
test_data.to_csv("test_data.csv")

In [ ]:
import transformers
import torch

from transformers.models.distilbert.modeling_distilbert import DistilBertForSequenceClassification
model = DistilBertForSequenceClassification.from_pretrained('content/sentiment-analysis-model')
model.to('cuda')

In [ ]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.utils.data import Dataset, DataLoader

# Define your custom dataset class
class MyDataset(Dataset):
    def __init__(self, file_path):
        self.tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
        self.labels = []
        self.sentences = []
        count = 0

        # Open the file and read the sentences and labels
        with open(file_path, "r") as f:
            for line in f:
              if '\t' not in line:
                count += 1
                continue
              label, sentence = line.strip().split("\t")
              print("Count:", count)
              self.labels.append(int(label))
              self.sentences.append(sentence)

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, index):
        # Tokenize the sentence and return it along with the label
        sentence = self.sentences[index]
        label = self.labels[index]
        encoding = self.tokenizer(sentence, truncation=True, padding=True, return_tensors="pt")
        return {"input_ids": encoding["input_ids"].squeeze(), "attention_mask": encoding["attention_mask"].squeeze(), "label": torch.tensor(label)}

# Load the model
# model_path = "path/to/extract"
# model = DistilBertForSequenceClassification.from_pretrained(model_path)

from transformers.models.distilbert.modeling_distilbert import DistilBertForSequenceClassification
model = DistilBertForSequenceClassification.from_pretrained('content/sentiment-analysis-model')
model.to('cuda')

# Load the test data
test_data_path = pd.read_csv("test_data.csv")
test_data = MyDataset(test_data_path)
test_loader = DataLoader(test_data, batch_size=16)

# Evaluate the model on the test data
model.eval()
with torch.no_grad():
    correct_predictions = 0
    total_predictions = 0
    for batch in test_loader:
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["label"]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_labels = torch.argmax(logits, axis=1)
        correct_predictions += (predicted_labels == labels).sum().item()
        total_predictions += labels.shape[0]
    accuracy = correct_predictions / total_predictions
    print(f"Test Accuracy: {accuracy:.2f}")


TypeError: ignored

In [ ]:
import torch
import transformers

# Load the fine-tuned DistilBERT model
# model_path = "path/to/fine-tuned/model"
# model = transformers.DistilBertForSequenceClassification.from_pretrained(model_path)

# Load the test data
test_data_path = "test_data.csv"
test_data = torch.utils.data.DataLoader(
    transformers.datasets.TextClassificationDataset(
        test_data_path,
        tokenizer=transformers.DistilBertTokenizer.from_pretrained("distilbert-base-uncased"),
        batch_size=16,
        max_length=512,
    )
)

# Evaluate the model on the test data
model.eval()
with torch.no_grad():
    correct_predictions = 0
    total_predictions = 0
    for batch in test_data:
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["label"]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_labels = torch.argmax(logits, axis=1)
        correct_predictions += (predicted_labels == labels).sum().item()
        total_predictions += labels.shape[0]
    accuracy = correct_predictions / total_predictions
    print(f"Test Accuracy: {accuracy:.2f}")


AttributeError: ignored

In [ ]:
import torch

test_data_path = "test_data.csv"
test_data = torch.utils.data.DataLoader(
    transformers.datasets.TextClassificationDataset(
        test_data_path,
        tokenizer=transformers.BertTokenizer.from_pretrained("bert-base-uncased"),
        batch_size=16,
        max_length=512,
    )
)

model.eval()
with torch.no_grad():
    correct_predictions = 0
    total_predictions = 0
    for batch in test_data:
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["label"]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_labels = torch.argmax(logits, axis=1)
        correct_predictions += (predicted_labels == labels).sum().item()
        total_predictions += labels.shape[0]
    accuracy = correct_predictions / total_predictions
    print(f"Test Accuracy: {accuracy:.2f}")


Testing

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [ ]:
def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in data:
        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Truncate/Pad sentence to max length
        #    (4) Map tokens to their IDs
        #    (5) Create attention mask
        #    (6) Return a dictionary of outputs
        encoded_sent = tokenizer.encode_plus(
            text=text_preprocessing(sent),  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=512,                  # Max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True      # Return attention mask
            )
        
        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

def text_preprocessing(s):
    """
    - Lowercase the sentence
    - Change "'t" to "not"
    - Remove "@name"
    - Isolate and remove punctuations except "?"
    - Remove other special characters
    - Remove stop words except "not" and "can"
    - Remove trailing whitespace
    """
    s = s.lower()
    # Change 't to 'not'
    s = re.sub(r"\'t", " not", s)
    # Remove @name
    s = re.sub(r'(@.*?)[\s]', ' ', s)
    # Isolate and remove punctuations except '?'
    s = re.sub(r'([\'\"\.\(\)\!\?\\\/\,])', r' \1 ', s)
    s = re.sub(r'[^\w\s\?]', ' ', s)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    # Remove stopwords except 'not' and 'can'
    s = " ".join([word for word in s.split()
                  if word not in stopwords.words('english')
                  or word in ['not', 'can']])
    # Remove trailing whitespace
    s = re.sub(r'\s+', ' ', s).strip()
    
    return s

In [ ]:
test_data = pd.read_csv("test_data.csv")

In [ ]:
test_data = test_data[["label", "text"]]
test_data.head()

In [ ]:
test_data.text

0         Great CD: My lovely Pat has one of the GREAT v...
1         One of the best game music soundtracks - for a...
2         Batteries died within a year ...: I bought thi...
3         works fine, but Maha Energy is better: Check o...
4         Great for the non-audiophile: Reviewed quite a...
                                ...                        
399995    Unbelievable- In a Bad Way: We bought this Tho...
399996    Almost Great, Until it Broke...: My son reciev...
399997    Disappointed !!!: I bought this toy for my son...
399998    Classic Jessica Mitford: This is a compilation...
399999    Comedy Scene, and Not Heard: This DVD will be ...
Name: text, Length: 400000, dtype: object

In [ ]:
import nltk
# Uncomment to download "stopwords"
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
print('Tokenizing data...')
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
test_inputs, test_masks = preprocessing_for_bert(test_data.text)

Tokenizing data...


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


KeyboardInterrupt: ignored

In [ ]:
import torch.nn.functional as F

def bert_predict(model, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    all_logits = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)
        all_logits.append(logits)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)

    # Apply softmax to calculate probabilities
    probs = F.softmax(all_logits, dim=1).cpu().numpy()

    return probs

In [ ]:
from transformers.models.distilbert.modeling_distilbert import DistilBertForSequenceClassification
pretrained_model = DistilBertForSequenceClassification.from_pretrained('content/sentiment-analysis-model')
pretrained_model.to('cuda')

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
#text = "Despite the initial skepticism surrounding its effectiveness, the new drug has shown promising results in treating a wide range of symptoms associated with the rare neurological disorder, which has left many patients and their families hopeful for the future."
#text = "While the product itself was impressive in terms of its versatility and durability, the instructions provided in the manual were convoluted and lacked clarity, making the assembly process much more challenging than it needed to be. Additionally, the customer service team was slow to respond to my inquiries and provided unhelpful solutions, leaving me feeling frustrated and dissatisfied with my overall experience."
#text = "I wouldn't never recommend this to everyone"
text = "Despite the rainy weather, I had a blast at the concert last night. The music was amazing and the crowd was energized. However, the venue was overcrowded and I had to stand in a cramped space for hours. Overall, it was a mixed experience."
#text = "I love this product, but it broke after a few uses. I'm disappointed, but I'm willing to try another one."
#text = "Despite the rain, she managed to enjoy her day at the beach"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = pretrained_model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
predictions

array([[0.34420267, 0.65579736]], dtype=float32)

# BERT Batch Fine-Tuning

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import bz2
import re
import os
import gc
import matplotlib.pyplot as plt
import torch

In [ ]:
train_file = bz2.BZ2File('train.ft.txt.bz2')
test_file = bz2.BZ2File('test.ft.txt.bz2')

In [ ]:
def load_extract(file):
    texts, labels = [], [] 
    for line in file:
        x = line.decode('utf-8')  # decode binary to string
        labels.append(int(x[9]) - 1)  # extract labels
        texts.append(x[10:].strip())  # extract texts
    print('Done !')
    return np.array(labels), texts

In [ ]:
train_labels, train_texts = load_extract(train_file)
test_labels, test_texts = load_extract(test_file)

Done !
Done !


In [ ]:
train_data = pd.DataFrame(data=train_labels, columns=['label'])
train_data['text'] = train_texts

In [ ]:
unique_labels = train_data['label'].unique()

# Initialize an empty DataFrame to store your filtered data
bert_train = pd.DataFrame()

# Loop through each label and select 5000 random records
for label in unique_labels:
    label_data = train_data.loc[train_data['label'] == label].sample(n=500000, random_state=42)
    bert_train = pd.concat([bert_train, label_data])

# Save the filtered data to a new file
bert_train.to_csv('bert_train.csv', index=False)

In [ ]:
dataset = pd.read_csv("bert_train.csv")

In [ ]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model = model.to('cuda')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier

In [ ]:
from torch.utils.data import Dataset, DataLoader
class CustomDataset(Dataset):
    def __init__(self, data, targets, tokenizer, max_len):
        self.data = data
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        data = str(self.data[index])
        target = self.targets[index]
        
        encoding = self.tokenizer.encode_plus(
            data,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(target, dtype=torch.long)
        }

In [ ]:
dataset.shape

(1000000, 2)

In [ ]:
import gc
from sklearn.model_selection import train_test_split
X = list(dataset["text"])
y = list(dataset["label"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

batch_size = 100000
num_batches = int(np.ceil(len(X_train) / batch_size))
batches_X_train = np.array_split(X_train, num_batches)
batches_y_train = np.array_split(y_train, num_batches)

# Fine-tune the model on each batch and save the checkpoint after each batch
for i in range(num_batches):
    print(f"Fine-tuning on batch {i+1}/{num_batches}")
    
    # Load model from previous checkpoint
    if i > 0:
        model = DistilBertForSequenceClassification.from_pretrained(f'model_checkpoint_{i-1}')
        model.to(device)
    
    # Tokenize batch
    #X_train_tokenized = tokenizer(batches_X_train[i], padding=True, truncation=True, max_length=512, return_tensors='pt')
    X_train_tokenized = tokenizer(batches_X_train[i], padding=True, truncation=True, max_length=512, return_tensors='pt',  batch_size=batch_size)

    X_train_tokenized = {k: v.to(device) for k, v in X_train_tokenized.items()}
    y_train_batch = torch.tensor(batches_y_train[i]).to(device)
    
    # Fine-tune model on batch
    optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
    loss_fn = torch.nn.CrossEntropyLoss()
    for epoch in range(2):
        running_loss = 0.0
        for j in range(0, len(X_train_tokenized['input_ids']), batch_size):
            input_ids_batch = X_train_tokenized['input_ids'][j:j+batch_size]
            attention_mask_batch = X_train_tokenized['attention_mask'][j:j+batch_size]
            optimizer.zero_grad()
            outputs = model(input_ids_batch, attention_mask=attention_mask_batch, labels=y_train_batch[j:j+batch_size])
            loss = loss_fn(outputs.logits, y_train_batch[j:j+batch_size])
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{2}, loss: {running_loss/len(X_train_tokenized['input_ids'])}")
    
    # Save checkpoint for current batch
    model.save_pretrained(f'model_checkpoint_{i}')
    
    # Clear memory
    del X_train_tokenized, y_train_batch
    gc.collect()

Fine-tuning on batch 1/8


ValueError: ignored